# This task is not quite ready as we don't have an open source route for simulating geometry that requires imprinting and merging. However this simulation can be carried out using Trelis.

# Heating Mesh Tally on CAD geometry made from Components

This constructs a reactor geometry from 3 Component objects each made from points.

The Component made include a breeder blanket, PF coil and a central column shield.

2D and 3D Meshes tally are then simulated to show nuclear heating, flux and tritium_production across the model.

This section makes the 3d geometry for the entire reactor from a input parameters.

In [ ]:
import paramak

my_reactor = paramak.BallReactor(
    inner_bore_radial_thickness=50,
    inboard_tf_leg_radial_thickness=55,
    center_column_shield_radial_thickness=50,
    divertor_radial_thickness=50,
    inner_plasma_gap_radial_thickness=50,
    plasma_radial_thickness=100,
    outer_plasma_gap_radial_thickness=50,
    firstwall_radial_thickness=1,
    blanket_radial_thickness=100,
    blanket_rear_wall_radial_thickness=10,
    elongation=2,
    triangularity=0.55,
    number_of_tf_coils=16,
    rotation_angle=180,
)

# TF and PF coils can be added with additional arguments.
# see the documentation for more details 
# https://paramak.readthedocs.io/en/main/paramak.parametric_reactors.html

my_reactor.solid

The next section defines the materials. This can be done using openmc.Materials or in this case strings that look up materials from the neutronics material maker.

In [ ]:
my_reactor.export_stp()

from IPython.display import FileLink
display(FileLink('blanket.stp'))
display(FileLink('pf_coil.stp'))
display(FileLink('center_column.stp'))
display(FileLink('Graveyard.stp'))

The next section defines the materials. This can be done using openmc.Materials or in this case strings that look up materials from the neutronics material maker.

In [ ]:
from neutronics_material_maker import Material

mat1 = Material(material_name='Li4SiO4',
                material_tag='blanket_material')

mat2 = Material(material_name='copper',
                material_tag='pf_coil_material')

mat3 = Material(material_name='WC',
                material_tag='center_column_material')

This next step makes a simple point source.

In [ ]:
import openmc

# initialises a new source object
source = openmc.Source()

# sets the location of the source to x=0 y=0 z=0
source.space = openmc.stats.Point((100, 0, 0))

# sets the direction to isotropic
source.angle = openmc.stats.Isotropic()

# sets the energy distribution to 100% 14MeV neutrons
source.energy = openmc.stats.Discrete([14e6], [1])

This next section combines the geometry with the materials and specifies a few mesh tallies

In [ ]:
neutronics_model = paramak.NeutronicsModel(
    geometry=my_reactor,
    cell_tallies=['heating', 'flux', 'TBR', 'spectra'],
    mesh_tally_2D=['heating', 'flux', 'tritium_production'],
    mesh_tally_3D=['heating', 'flux', 'tritium_production'],
    source=source,
    simulation_batches=2,
    simulation_particles_per_batch=10000,
    materials={
        'blanket_material': mat1,
        'pf_coil_material': mat2,
        'center_column_material': mat3,
    }
)


# You will need to have Trelis installed to run this command
neutronics_model.simulate(method='trelis')

The next section produces download links for:

- vtk files that contain the 3D mesh results (open with Paraview)
- png images that show the resuls of the 2D mesh tally

In [ ]:
from IPython.display import FileLink
display(FileLink('heating_on_3D_mesh.vtk'))
display(FileLink('flux_on_3D_mesh.vtk'))
display(FileLink('tritium_production_on_3D_mesh.vtk'))
display(FileLink('flux_on_2D_mesh_xy.png'))
display(FileLink('flux_on_2D_mesh_xz.png'))
display(FileLink('flux_on_2D_mesh_yz.png'))
display(FileLink('heating_on_2D_mesh_xy.png'))
display(FileLink('heating_on_2D_mesh_xz.png'))
display(FileLink('heating_on_2D_mesh_yz.png'))
display(FileLink('tritium_production_on_2D_mesh_yz.png'))
display(FileLink('tritium_production_on_2D_mesh_xz.png'))
display(FileLink('tritium_production_on_2D_mesh_yz.png'))